In [18]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig, get_linear_schedule_with_warmup
import torch 
import torch.nn as nn

# Pipeline Inference

In [11]:
device = torch.device("cpu") 

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC", num_labels=3)
model.eval()
torch.save(model.state_dict(), f"best_chkpt_origin.pt")

In [13]:
test_model1 = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC", num_labels=3) 
checkpoint1 = torch.load("best_chkpt_origin.pt") 
print(test_model1.load_state_dict(checkpoint1))
test_model1.to(device) 
test_model1.eval() 
print()

<All keys matched successfully>



In [14]:
my_pipeline = pipeline("sentiment-analysis", model=test_model1, tokenizer="snunlp/KR-FinBert-SC")
print(my_pipeline("그러나 이 중개인은 주식에 대해 초과 추천을 했다."))
print(my_pipeline("피스카는 영국 전역에서 지역사회 프로젝트를 수행할 예정이며 선정된 복구 프로젝트에 도움을 주는 데 관심이 있는 현지 자원봉사자들을 찾고 있다."))

[{'label': 'neutral', 'score': 0.9998950958251953}]
[{'label': 'neutral', 'score': 0.6948619484901428}]


# Custom Inference

In [24]:
labels = my_pipeline.model.config.id2label
labels

{0: 'negative', 1: 'neutral', 2: 'positive'}

In [29]:
sentences = ["그러나 이 중개인은 주식에 대해 초과 추천을 했다.", "피스카는 영국 전역에서 지역사회 프로젝트를 수행할 예정이며 선정된 복구 프로젝트에 도움을 주는 데 관심이 있는 현지 자원봉사자들을 찾고 있다."]

device = torch.device("cpu") 
test_model1 = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC", num_labels=3) 
checkpoint1 = torch.load("best_chkpt_origin.pt") 
print(test_model1.load_state_dict(checkpoint1))
test_model1.to(device) 
test_model1.eval() 

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

for sentence in sentences:
    encoded_input = tokenizer(sentence, return_tensors="pt") 
    with torch.no_grad():
        output = test_model1(**encoded_input) 
        probs = nn.Softmax()(output.logits)  
        score = torch.max(probs, dim=1)
        print(f"label: {labels[score.indices.item()]}, score:{score.values.item()}")

<All keys matched successfully>
label: neutral, score:0.9998950958251953
label: neutral, score:0.6948619484901428


/tmp/ipykernel_217/1357513801.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = nn.Softmax()(output.logits)
